In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

In [2]:
UNIT_DICT = {'QB':1,'RB':1,'WR':1,'TE':1,'C':1,'G':1,'T':1,'DT':2,'DE':2,'LB':2,'DB':2, 'K':1}
FIRST_YEAR = 2000
CURRENT_YEAR = 2021

In [3]:
def get_train_df():
    df = pd.read_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR-1) + ' Reference Data.csv', encoding='utf-8')
    
    # Drop Non Numerical Columns
    non_num_cols = ['ID', 'Name', 'Team', 'Year']
    df.drop(columns=non_num_cols, inplace=True)

    # Encode Positions To Offense / Def
    for i in df.index:
        df.loc[i, 'Position'] = UNIT_DICT[df.loc[i, 'Position']]
        
    # Drop Null
    for i in df[df.isnull().any(axis=1)].index:
        print('Null Row: ', i)
    df.dropna(inplace=True)

    return df

In [4]:
train_df = get_train_df()
train_df

,Position,Pick,Team PF,Team Yds,Team Tot Yds & TOPly,Team Tot Yds & TOY/P,Team Tot Yds & TOTO,Team FL,Team 1stD,Team PassingCmp,...,Def. Rank RushingYds,Def. Rank RushingTD,Def. Rank RushingY/A,Def. Rank Sc%,Def. Rank TO%,Def. Rank Average DriveStart,Def. Rank Average DriveTime,Def. Rank Average DrivePlays,Def. Rank Average DriveYds,Def. Rank Average DrivePts
0,1,1,279.0,5169.0,1049.0,4.9,30.0,14.0,312.0,356.0,...,32.0,27.0,25.0,9.0,27.0,12.0,18.0,15.0,28.0,22.0
1,2,2,266.0,4395.0,885.0,5.0,21.0,8.0,248.0,298.0,...,31.0,17.0,26.0,4.0,15.0,31.0,31.0,29.0,29.0,30.0
2,2,3,341.0,5549.0,1021.0,5.4,23.0,8.0,313.0,344.0,...,21.0,14.0,10.0,7.0,25.0,3.0,21.0,28.0,30.0,24.0
3,1,4,341.0,5416.0,1012.0,5.4,33.0,16.0,311.0,376.0,...,20.0,28.0,4.0,8.0,29.0,25.0,19.0,17.0,20.0,25.0
4,1,5,306.0,4960.0,1022.0,4.9,26.0,8.0,315.0,371.0,...,27.0,21.0,22.0,1.0,28.0,27.0,20.0,20.0,26.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,1,27,308.0,4752.0,999.0,4.8,22.0,6.0,268.0,272.0,...,17.0,25.0,20.0,12.0,6.0,4.0,13.0,23.0,22.0,17.0
663,2,28,423.0,5726.0,979.0,5.8,28.0,11.0,327.0,338.0,...,18.0,18.0,26.0,8.0,27.0,14.0,24.0,28.0,25.0,23.0
664,1,29,396.0,5586.0,1085.0,5.1,18.0,7.0,331.0,320.0,...,7.0,2.0,14.0,30.0,14.0,1.0,9.0,6.0,5.0,4.0
665,2,30,392.0,5296.0,1011.0,5.2,22.0,9.0,294.0,304.0,...,10.0,7.0,22.0,26.0,4.0,2.0,6.0,11.0,17.0,12.0


In [5]:
def get_test_df():
    df = pd.read_csv(str(CURRENT_YEAR) + ' Raw Test Data.csv', encoding='utf-8')
    df = df.drop(columns=['Round', 'Team'])
    
    # Convert Numerical
    for i in df.columns:
        if 'DriveStart' in i and 'Rank' not in i:
            for j in df.index:
                if pd.isnull(df.loc[j, i]) == False:
                    df.loc[j, i] = df.loc[j, i].split(' ')[1]
        elif 'DriveTime' in i and 'Rank' not in i:
            for j in df.index:
                if pd.isnull(df.loc[j, i]) == False:
                    df.loc[j, i] = round(int(df.loc[j, i].split(':')[0])+int(df.loc[j, i].split(':')[1])/60, 2)
        else:
            df[i] = pd.to_numeric(df[i], downcast = 'float')
        
    return df

In [6]:
test_df = get_test_df()
test_df

,Pick,Team PF,Team Yds,Team Tot Yds & TOPly,Team Tot Yds & TOY/P,Team Tot Yds & TOTO,Team FL,Team 1stD,Team PassingCmp,Team PassingAtt,...,Def. Rank RushingYds,Def. Rank RushingTD,Def. Rank RushingY/A,Def. Rank Sc%,Def. Rank TO%,Def. Rank Average DriveStart,Def. Rank Average DriveTime,Def. Rank Average DrivePlays,Def. Rank Average DriveYds,Def. Rank Average DrivePts
0,1.0,306.0,5218.0,997.0,5.2,25.0,9.0,310.0,387.0,616.0,...,30.0,29.0,27.0,4.0,26.0,29.0,22.0,13.0,26.0,29.0
1,2.0,243.0,4479.0,948.0,4.7,19.0,5.0,269.0,292.0,499.0,...,12.0,14.0,7.0,8.0,20.0,26.0,26.0,21.0,23.0,24.0
2,3.0,376.0,5922.0,1046.0,5.7,31.0,14.0,350.0,371.0,570.0,...,7.0,4.0,6.0,20.0,23.0,25.0,3.0,5.0,4.0,8.0
3,4.0,396.0,5895.0,1078.0,5.5,18.0,7.0,366.0,408.0,628.0,...,6.0,13.0,14.0,14.0,16.0,15.0,11.0,16.0,27.0,21.0
4,5.0,311.0,5116.0,1040.0,4.9,24.0,13.0,318.0,372.0,581.0,...,29.0,7.0,31.0,11.0,25.0,18.0,7.0,8.0,20.0,20.0
5,6.0,404.0,5424.0,1021.0,5.3,20.0,7.0,345.0,370.0,559.0,...,16.0,18.0,18.0,29.0,1.0,6.0,4.0,6.0,14.0,5.0
6,7.0,377.0,5603.0,991.0,5.7,21.0,8.0,350.0,374.0,582.0,...,28.0,32.0,15.0,3.0,31.0,11.0,28.0,26.0,32.0,32.0
7,8.0,350.0,5592.0,993.0,5.6,21.0,5.0,335.0,373.0,550.0,...,20.0,18.0,28.0,6.0,11.0,23.0,24.0,28.0,24.0,26.0
8,9.0,323.0,5369.0,1030.0,5.2,32.0,9.0,308.0,317.0,556.0,...,25.0,28.0,29.0,9.0,30.0,30.0,11.0,11.0,11.0,19.0
9,10.0,395.0,5949.0,1113.0,5.3,26.0,13.0,371.0,413.0,639.0,...,31.0,25.0,30.0,12.0,15.0,28.0,17.0,12.0,19.0,25.0


In [7]:
# Split More
X_train = train_df.drop('Position', axis=1)
Y_train = train_df[['Position']]
X_test = test_df.copy()
X_train.shape, Y_train.shape, X_test.shape

((667, 103), (667, 1), (32, 103))

In [8]:
train_dataset = pd.concat([Y_train, X_train], axis=1)
train_dataset.to_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR) + ' Train Dataset.csv', encoding='utf-8-sig', index=False)

In [9]:
# Scale
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
X_train = pd.DataFrame(X_train)
X_train.columns = train_df.drop('Position',axis=1).columns

X_test = pd.DataFrame(X_test)
X_test.columns = test_df.columns

In [11]:
X_train

,Pick,Team PF,Team Yds,Team Tot Yds & TOPly,Team Tot Yds & TOY/P,Team Tot Yds & TOTO,Team FL,Team 1stD,Team PassingCmp,Team PassingAtt,...,Def. Rank RushingYds,Def. Rank RushingTD,Def. Rank RushingY/A,Def. Rank Sc%,Def. Rank TO%,Def. Rank Average DriveStart,Def. Rank Average DriveTime,Def. Rank Average DrivePlays,Def. Rank Average DriveYds,Def. Rank Average DrivePts
0,-1.682749,-0.910269,-0.225545,0.829769,-0.662630,0.511238,0.864167,0.211392,0.558187,1.279859,...,1.616902,1.129330,0.888626,-0.685687,1.081425,-0.522302,0.126558,-0.245221,1.156068,0.484183
1,-1.573694,-1.092774,-1.419134,-2.645489,-0.476064,-0.788619,-0.751906,-1.481967,-0.645963,-1.062371,...,1.507412,0.042684,0.998836,-1.227575,-0.223721,1.555265,1.544369,1.288516,1.265118,1.353498
2,-1.464639,-0.039864,0.360455,0.236432,0.270198,-0.499762,-0.751906,0.237851,0.309052,0.510513,...,0.412515,-0.283310,-0.764535,-0.902442,0.863901,-1.506412,0.453745,1.178964,1.374169,0.701512
3,-1.355584,-0.039864,0.155355,0.045717,0.270198,0.944523,1.402859,0.184933,0.973411,1.125990,...,0.303026,1.237995,-1.425800,-0.794065,1.298950,0.899192,0.235621,-0.026115,0.283662,0.810176
4,-1.246529,-0.531222,-0.547845,0.257623,-0.662630,-0.066476,-0.751906,0.290768,0.869605,1.262762,...,1.069454,0.477343,0.557994,-1.552708,1.190188,1.117883,0.344683,0.302543,0.937966,1.570827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,1.152680,-0.503144,-0.868602,-0.229761,-0.849195,-0.644190,-1.290598,-0.952792,-1.185754,-1.113661,...,-0.025444,0.912001,0.337572,-0.360554,-1.202580,-1.397067,-0.418754,0.631201,0.501764,-0.059138
663,1.261735,1.111317,0.633407,-0.653573,1.016461,0.222381,0.056130,0.608273,0.184485,0.083099,...,0.084046,0.151349,0.998836,-0.794065,1.081425,-0.303610,0.780933,1.178964,0.828916,0.592848
664,1.370789,0.732269,0.417513,1.592631,-0.289498,-1.221904,-1.021252,0.714108,-0.189216,-0.104963,...,-1.120341,-1.587286,-0.323692,1.590242,-0.332483,-1.725103,-0.855003,-1.231194,-1.352097,-1.471774
665,1.479844,0.676114,-0.029698,0.024526,-0.102933,-0.644190,-0.482561,-0.264865,-0.521396,-0.241736,...,-0.791872,-1.043963,0.557994,1.156732,-1.420105,-1.615758,-1.182191,-0.683431,-0.043489,-0.602460


In [12]:
Y_train

,Position
0,1
1,2
2,2
3,1
4,1
...,...
662,1
663,2
664,1
665,2


In [13]:
X_test

,Pick,Team PF,Team Yds,Team Tot Yds & TOPly,Team Tot Yds & TOY/P,Team Tot Yds & TOTO,Team FL,Team 1stD,Team PassingCmp,Team PassingAtt,...,Def. Rank RushingYds,Def. Rank RushingTD,Def. Rank RushingY/A,Def. Rank Sc%,Def. Rank TO%,Def. Rank Average DriveStart,Def. Rank Average DriveTime,Def. Rank Average DrivePlays,Def. Rank Average DriveYds,Def. Rank Average DrivePts
0,-1.682749,-0.531222,-0.149982,-0.272142,-0.102933,-0.210905,-0.482561,0.158475,1.201784,1.279859,...,1.397923,1.346660,1.109047,-1.227575,0.972663,1.336574,0.562808,-0.464326,0.937966,1.244834
1,-1.573694,-1.415666,-1.289597,-1.310481,-1.035761,-1.077476,-1.559943,-0.926334,-0.770530,-0.720440,...,-0.572892,-0.283310,-1.095168,-0.794065,0.320090,1.008537,0.999057,0.412095,0.610814,0.701512
2,-1.464639,0.451493,0.935660,0.766197,0.829895,0.655666,0.864167,1.216824,0.869605,0.493417,...,-1.120341,-1.369957,-1.205378,0.506466,0.646377,0.899192,-1.509378,-1.340747,-1.461148,-1.037117
3,-1.355584,0.732269,0.894023,1.444296,0.456764,-1.221904,-1.021252,1.640164,1.637769,1.485018,...,-1.229831,-0.391975,-0.323692,-0.143799,-0.114959,-0.194265,-0.636879,-0.135668,1.047017,0.375519
4,-1.246529,-0.461028,-0.307276,0.639054,-0.662629,-0.355333,0.594822,0.370145,0.890366,0.681479,...,1.288433,-1.043963,1.549890,-0.468932,0.863901,0.133772,-1.073128,-1.012089,0.283662,0.266855
5,-1.137474,0.844580,0.167692,0.236432,0.083633,-0.933047,-1.021252,1.084531,0.848844,0.305354,...,-0.134933,0.151349,0.117151,1.481864,-1.746391,-1.178375,-1.400315,-1.231194,-0.370641,-1.363110
6,-1.028419,0.465532,0.443728,-0.399285,0.829895,-0.788619,-0.751906,1.216824,0.931888,0.698575,...,1.178943,1.672654,-0.213482,-1.335953,1.516474,-0.631647,1.217182,0.959858,1.592270,1.570827
7,-0.919364,0.086485,0.426765,-0.356904,0.643329,-0.788619,-1.559943,0.819943,0.911127,0.151485,...,0.303026,0.151349,1.219258,-1.010820,-0.658769,0.680500,0.780933,1.178964,0.719865,0.918841
8,-0.810309,-0.292563,0.082876,0.427148,-0.102933,0.800095,-0.482561,0.105557,-0.251500,0.254065,...,0.850474,1.237995,1.329469,-0.685687,1.407712,1.445920,-0.636879,-0.683431,-0.697793,0.158190
9,-0.701254,0.718231,0.977296,2.185967,0.083633,-0.066476,0.594822,1.772458,1.741575,1.673080,...,1.507412,0.912001,1.439679,-0.360554,-0.223721,1.227228,0.017496,-0.573878,0.174612,0.810176


In [14]:
X_train.to_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR) + ' X Train.csv', encoding='utf-8-sig', index=False)
Y_train.to_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR) + ' Y Train.csv', encoding='utf-8-sig', index=False)
X_test.to_csv(str(FIRST_YEAR) + '-' + str(CURRENT_YEAR) + ' X Test.csv', encoding='utf-8-sig', index=False)